In [1]:
import os
import re
import pandas as pd

In [2]:
# Functions to preprocess each file 
# The title and text of the file follow the format of the Naver web novel

def link_to_id(text):
    """
    (ko) 네이버 웹소설에서 등장인물을 표시하는 사진 링크를 가져와, id를 추출하는 전처리를 수행하는 함수.
    사진 링크가 있는 웹소설만을 데이터로 사용할 때 쓸 수 있는 함수로, 네이버 웹소설 형식에 맞춰져 있습니다.
    
    (en) It's a function that takes a photo link which displays a character in a Naver web novel, 
    and performs a preprocessing process to extract ids used. 

    논항 Args:
        text: 사진 링크가 포함된 네이버 웹소설 각 회차 컨텐츠(text).
              each episode of Naver Web Novel content (text) with photo link.

    리턴값 Returns:
        mod_text: 추출한 id로 링크를 대체한 수정된 텍스트
                  modified text that replaces the link with the extracted id
        ids: 추출한 id 리스트 (중복 있음). 
             the extracted id list (with duplicate).
    """
    pattern = re.compile(r'\.net/(\d+_\d+)/') # id extraction pattern

    matches = re.finditer(pattern, text)
    mod_text = text  # copy text to modify
    ids = list()

    for match in matches:
        if match:
            id_ = match.group(1)
            mod_text = re.sub(fr'https://.+?/{id_}.+=w80_2', f'{id_}: ', mod_text) #링크를 아이디로 변환
            ids.append(id_) 

    return mod_text, ids #string tuple
    
def preprocessing(text):
    return re.sub('\n+', '\n', text.strip())    
    

In [3]:
# 데이터프레임으로 파일 정보 저장
# | 소설 이름 | 회차 | 회차 이름 | 회차 내용 (id는 전처리됨) | 등장인물 id |

# Save file information as a data frame
# | Novel Name | Round | Round Name | Round Content (id preprocessed) | Character ids |

file_path = './Novels_text/Labeled/'  
novdf = pd.DataFrame({'novtitle':[], 'r':[], 'rtitle':[], 'rcontent':[], 'ids':[]})

for filename in os.listdir(file_path):
    filepath = os.path.join(file_path, filename)

    if os.path.isfile(filepath):
        with open(filepath, 'r', encoding='utf8') as f: 
            match = re.search(r'(\d+)[.] (.+), (.+)  네이버웹소설.txt', filename)

            if match:
                r, rtitle, novtitle = match.groups()               
                content, id_ = link_to_id(preprocessing(f.read())) # 정의해둔 전처리함수로 읽어들인 파일 전처리를 수행합니다.
                
                # Create a new DataFrame for the current row
                df = pd.DataFrame({'novtitle': [novtitle], 'r': [r], 'rtitle': [rtitle], 'rcontent': [content], 'ids': [id_]})
                
                # Concatenate the new DataFrame with the existing DataFrame
                novdf = pd.concat([novdf, df], ignore_index=True)
                novdf['r'] = novdf['r'].astype(int)

# 소설 제목과 회차 순서대로 정렬 Sort in order of fiction titles and rounds
novdf = novdf.sort_values(by=['novtitle', 'r'], ascending=[True, True]).reset_index(drop=True)
novlist = list(set(novdf['novtitle']))

print(novlist)
novdf

['남편이 사랑에 빠졌다', '그날 밤 사정', '결혼 먼저 해버린 부부']


,novtitle,r,rtitle,rcontent,ids
0,결혼 먼저 해버린 부부,1,돌아온 남편,팡- 팡-\n사진 작가가 카메라 셔터를 누름과 동시에 야외에 설치되어 있는 조명이 ...,"[20221028_233, 20221028_233, 20221028_233, 202..."
1,결혼 먼저 해버린 부부,2,보고 싶었습니까,그러나 곧이어 들려온 도준의 음성으로 인해 태연의 바람은 와장창 깨졌다.\n2022...,"[20221102_115, 20221028_233, 20221102_115, 202..."
2,결혼 먼저 해버린 부부,3,미리 입을 맞췄으면 좋겠는데,굳이 왜. \n설마 이 공간에도 도청 장치가 있어 누군가 대화를 듣고 있는 건가 싶...,"[20221102_115, 20221102_115, 20221102_169, 202..."
3,결혼 먼저 해버린 부부,4,사랑하는 남자 앞에서는,20221102_115: “당연히 당신이 도와줄 거라고 생각했는데.”\n202211...,"[20221102_115, 20221102_169, 20221102_115, 202..."
4,결혼 먼저 해버린 부부,5,무슨 짓을 할지 모릅니다,20221102_115: “내가 무슨 짓을 할지 모릅니다.”\n도준의 나직한 경고에...,"[20221102_115, 20221102_115, 20221102_115, 202..."
...,...,...,...,...,...
253,남편이 사랑에 빠졌다,41,내 몸에서 깨어나지 마,20220405_87: “내가?”\n경준이 매우 어이없다는 듯 얼굴을 찌푸렸다. 물...,"[20220405_87, 20220331_168, 20220331_168, 2022..."
254,남편이 사랑에 빠졌다,42,윤서하가 깨어나는 방법,경준은 다급하게 떨어진 산소호흡기를 주웠다. 동시에 조서하가 갑자기 실 끊어진 마리...,"[20220405_87, 20220405_87, 20220405_87, 202204..."
255,남편이 사랑에 빠졌다,43,N각관계,20220331_25: “우리 디자이너?”\n해선이 호기심 어린 표정으로 서하의 얼...,"[20220331_25, 20220331_25, 20220419_206, 20220..."
256,남편이 사랑에 빠졌다,44,들었을까,"나가지 말자.\n서하는 처음에 당연히 그렇게 생각했다. 류경준, 이승오와 셋이서 식...","[20220331_168, 20220419_206, 20220331_168, 202..."


In [30]:
def nov_concat(novdf):
    '''
    (ko) 소설 별 인물과 회차를 누적해서 데이터프레임으로 저장합니다.
    각 회차의 구분을 위해 회차 간 구분자를 '\n***\n'으로 설정합니다. '***'은 하나의 회차 내에서도 장면 구분을 위해 쓰이는 구분자이므로, 이후 scene을 나눌 때 일괄적으로 처리할 수 있습니다. 
    작가가 한 회차의 마지막 문장과 다음 회차의 첫 문장을 동일하게 반복하는 경우를 드물지 않게 볼 수 있습니다. 후작업에서 참고 바랍니다. 
    
    (en) It accumulates characters and episodes by novel and stores them as data frames.
    Note that the delimiter between rounds to '\n***\n' to separate each round. 

    Arg:
        novdf: | 소설 이름 | 회차 | 회차 이름 | 회차 내용 (id는 전처리됨) | 등장인물 id | 으로 저장한 각 파일의 정보 데이터프레임
               | Novel Name | Round | Round Name | Round Content (id preprocessed) | Character ids | (dataframe)

    Return:
        nov_cont: | 소설 이름 | 회차 별 내용을 누적한 전체 내용 | 누적된 id | 으로 저장한 각 소설의 정보 데이터프레임
                  | Title | Content | ID | (dataframe)
    '''
    novs = novdf.groupby('novtitle').agg({'rcontent':'\n***\n'.join, 'ids': lambda x: list(set(sum(x, [])))}).reset_index()
    novs.columns = ['Title', 'Content', 'ID']
    return novs

# print(nov_concat(novdf)['rcontent'][0])
accum_novdf = nov_concat(novdf)
accum_novdf

,Title,Content,ID
0,결혼 먼저 해버린 부부,팡- 팡-\n사진 작가가 카메라 셔터를 누름과 동시에 야외에 설치되어 있는 조명이 ...,"[20221111_246, 20221110_40, 20221111_270, 2022..."
1,그날 밤 사정,20221105_83: “오늘은 배란일이어서 그냥 넘기면 안 돼요.”\n수연은 건조...,"[20221111_110, 20221105_215, 20221107_80, 2022..."
2,남편이 사랑에 빠졌다,서하는 아주 긴 꿈속을 헤매다가 어느 순간 정신을 차렸다. 그러나 눈을 뜨지는 못했...,"[20220331_25, 20220405_193, 20220419_206, 2022..."


In [52]:
# Check Special Characters: To check quotes types
# 특수 문자 확인: 인용문의 종류 체크

def extract_special_characters(text):
    pattern = re.compile(r'[^\w\s|_|:|!|\.|?|…|－|―|]')
    special_characters = list(set(re.findall(pattern, text)))
    
    return special_characters

print(*extract_special_characters(accum_novdf['Content'][1]), sep='\t')

”	*	‘	>	“	]	[	’	<	,
